In [ ]:
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
import numpy as np

file_path = Path('../data/paper_trading_journal.xlsx')
df_trades = pd.read_excel(file_path, sheet_name=0)
df_tickers = pd.read_excel(file_path, sheet_name=1)

df_trades.dropna(subset=['Total Pts'], inplace=True)

total_pts_counts = df_trades.groupby('Total Pts').size()

def market_cap_to_numeric(value):
    if isinstance(value, str): 
        if 'B' in value:
            return float(value.replace('B', '')) * 1e9
        elif 'M' in value:
            return float(value.replace('M', '')) * 1e6
        else:
            return float(value)
    else:
        return value

df_trades['Market Cap Numeric'] = df_trades['Market Cap (as of buy)'].apply(market_cap_to_numeric)

df_trades = df_trades[df_trades['Market Cap Numeric'] >= 100e6]

df_trades_sorted = df_trades.sort_values('Market Cap Numeric')

df_trades_sorted_desc = df_trades.sort_values('Market Cap Numeric', ascending=False)

def categorize_market_cap(value):
    if pd.isna(value):
        return 'Not Available'
    elif value > 200e9:
        return 'Mega Cap: Over 200B'
    elif value > 10e9:
        return 'Large Cap: 10B to 200B'
    elif value > 2e9:
        return 'Mid Cap: 2B to 10B'
    elif value > 500e6:
        return 'Small Cap: 500M to 2B'
    elif value > 50e6:
        return 'Micro Cap 50M to 500M'
    else:
        return 'Nano Cap: Under 50M'

df_trades_sorted_desc['Market Cap Category'] = df_trades_sorted_desc['Market Cap Numeric'].apply(categorize_market_cap)
market_cap_counts = df_trades_sorted_desc['Market Cap Category'].value_counts()

categories_order = ['Mega Cap: Over 200B', 'Large Cap: 10B to 200B', 'Mid Cap: 2B to 10B', 'Small Cap: 500M to 2B', 'Micro Cap 50M to 500M', 'Nano Cap: Under 50M', 'Not Available']

market_cap_counts_ordered = market_cap_counts.reindex(categories_order)

df_trades['Percentage'] = pd.to_numeric(df_trades['Percentage'], errors='coerce')

def format_market_cap(x, pos):
    """Custom formatter for market cap values."""
    if x >= 1e9:
        return '{:.1f}B'.format(x * 1e-9)
    elif x >= 1e6:
        return '{:.1f}M'.format(x * 1e-6)
    elif x >= 1e3:
        return '{:.1f}K'.format(x * 1e-3)
    else:
        return str(x)

df_trades_complete = df_trades.dropna(subset=['Date Sold']).copy()

df_trades_complete['Date Bought'] = pd.to_datetime(df_trades_complete['Date Bought'])
df_trades_complete['Date Sold'] = pd.to_datetime(df_trades_complete['Date Sold'])

df_trades_complete['Business Days'] = df_trades_complete.apply(
    lambda row: np.busday_count(row['Date Bought'].date(), row['Date Sold'].date()), axis=1)

average_trade_duration_business_days = round(df_trades_complete['Business Days'].mean(), 1)

print("Average Trade Duration in Business Days:", average_trade_duration_business_days)

trades_won_count = df_trades[df_trades['Percentage'] > 0].shape[0]

trades_lost_count = df_trades[df_trades['Percentage'] <= 0].shape[0]

total_trades = trades_won_count + trades_lost_count
win_rate = (trades_won_count / total_trades) * 100 if total_trades > 0 else 0

print(f"Total Number of Trades Won: {trades_won_count}")
print(f"Total Number of Trades Lost: {trades_lost_count}")
print(f"Win Rate: {win_rate:.2f}%")

average_percentage_won = df_trades[df_trades['Percentage'] > 0]['Percentage'].mean() * 100

average_percentage_lost = df_trades[df_trades['Percentage'] <= 0]['Percentage'].mean() * 100

print(f"Average Percentage of Trades Won: {average_percentage_won:.2f}%")
print(f"Average Percentage of Trades Lost: {average_percentage_lost:.2f}%")

df_trades['Market Cap Category'] = df_trades['Market Cap Numeric'].apply(categorize_market_cap)
df_trades_completed = df_trades.dropna(subset=['Date Sold'])

completed_trades_per_category = df_trades_completed['Market Cap Category'].value_counts()

categories_order = ['Mega Cap: Over 200B', 'Large Cap: 10B to 200B', 'Mid Cap: 2B to 10B', 'Small Cap: 500M to 2B', 'Micro Cap 50M to 500M', 'Nano Cap: Under 50M', 'Not Available']
completed_trades_per_category_reindexed = completed_trades_per_category.reindex(categories_order, fill_value=0)

print()
print("Completed trades by Market Cap")
print(completed_trades_per_category_reindexed.to_string(header=False))

completed_trades_in_range = df_trades_complete[(df_trades_complete['Market Cap Numeric'] > 200e6) & (df_trades_complete['Market Cap Numeric'] <= 1e9)]

count_completed_trades_in_range = completed_trades_in_range.shape[0]

completed_trades_won = completed_trades_in_range[completed_trades_in_range['Percentage'] > 0]
completed_trades_lost = completed_trades_in_range[completed_trades_in_range['Percentage'] <= 0]
win_rate_completed = (len(completed_trades_won) / len(completed_trades_in_range)) * 100 if len(completed_trades_in_range) > 0 else 0

average_percentage_won_completed = completed_trades_won['Percentage'].mean() if len(completed_trades_won) > 0 else 0
average_percentage_lost_completed = completed_trades_lost['Percentage'].mean() if len(completed_trades_lost) > 0 else 0

print()
print(f"Number of completed trades with market cap between $200M and $1B: {count_completed_trades_in_range}")
print(f"Win rate for completed trades with market cap between $200M and $1B: {win_rate_completed:.2f}%")
print(f"Average percentage of completed trades won: {average_percentage_won_completed * 100:.2f}%")
print(f"Average percentage of completed trades lost: {average_percentage_lost_completed * 100:.2f}%")

completed_trades_over_1b = df_trades_complete[df_trades_complete['Market Cap Numeric'] > 1e9]

count_completed_trades_over_1b = completed_trades_over_1b.shape[0]

completed_trades_won_over_1b = completed_trades_over_1b[completed_trades_over_1b['Percentage'] > 0]
completed_trades_lost_over_1b = completed_trades_over_1b[completed_trades_over_1b['Percentage'] <= 0]
win_rate_completed_over_1b = (len(completed_trades_won_over_1b) / len(completed_trades_over_1b)) * 100 if len(completed_trades_over_1b) > 0 else 0

average_percentage_won_completed_over_1b = completed_trades_won_over_1b['Percentage'].mean() if len(completed_trades_won_over_1b) > 0 else 0
average_percentage_lost_completed_over_1b = completed_trades_lost_over_1b['Percentage'].mean() if len(completed_trades_lost_over_1b) > 0 else 0

print()
print(f"Number of completed trades with market cap over $1B: {count_completed_trades_over_1b}")
print(f"Win rate for completed trades with market cap over $1B: {win_rate_completed_over_1b:.2f}%")
print(f"Average percentage of completed trades won: {average_percentage_won_completed_over_1b * 100:.2f}%")
print(f"Average percentage of completed trades lost: {average_percentage_lost_completed_over_1b * 100:.2f}%")

trades_sum_exactly_8_ML = df_trades_complete[(df_trades_complete['Medium'] + df_trades_complete['Long']) == 8]

count_trades_sum_exactly_8_ML = trades_sum_exactly_8_ML.shape[0]

trades_won_sum_exactly_8_ML = trades_sum_exactly_8_ML[trades_sum_exactly_8_ML['Percentage'] > 0]
trades_lost_sum_exactly_8_ML = trades_sum_exactly_8_ML[trades_sum_exactly_8_ML['Percentage'] <= 0]

win_rate_sum_exactly_8_ML = (len(trades_won_sum_exactly_8_ML) / len(trades_sum_exactly_8_ML)) * 100 if len(trades_sum_exactly_8_ML) > 0 else 0

average_percentage_won_sum_exactly_8_ML = trades_won_sum_exactly_8_ML['Percentage'].mean() * 100 if len(trades_won_sum_exactly_8_ML) > 0 else 0
average_percentage_lost_sum_exactly_8_ML = trades_lost_sum_exactly_8_ML['Percentage'].mean() * 100 if len(trades_lost_sum_exactly_8_ML) > 0 else 0

print()
print(f"Number of completed trades with Medium + Long sum = 8: {count_trades_sum_exactly_8_ML}")
print(f"Win rate for trades with Medium + Long sum = 8: {win_rate_sum_exactly_8_ML:.2f}%")
print(f"Average percentage of won trades: {average_percentage_won_sum_exactly_8_ML:.2f}%")
print(f"Average percentage of lost trades: {average_percentage_lost_sum_exactly_8_ML:.2f}%")

trades_sum_7_or_greater = df_trades_complete[(df_trades_complete['Medium'] + df_trades_complete['Long']) >= 7]

count_trades_sum_7_or_greater = trades_sum_7_or_greater.shape[0]

trades_won_sum_7_or_greater = trades_sum_7_or_greater[trades_sum_7_or_greater['Percentage'] > 0]
trades_lost_sum_7_or_greater = trades_sum_7_or_greater[trades_sum_7_or_greater['Percentage'] <= 0]

win_rate_sum_7_or_greater = (len(trades_won_sum_7_or_greater) / len(trades_sum_7_or_greater)) * 100 if len(trades_sum_7_or_greater) > 0 else 0

average_percentage_won_sum_7_or_greater = trades_won_sum_7_or_greater['Percentage'].mean() * 100 if len(trades_won_sum_7_or_greater) > 0 else 0
average_percentage_lost_sum_7_or_greater = trades_lost_sum_7_or_greater['Percentage'].mean() * 100 if len(trades_lost_sum_7_or_greater) > 0 else 0

print()
print(f"Number of completed trades with Medium + Long sum ≥ 7: {count_trades_sum_7_or_greater}")
print(f"Win rate for trades with Medium + Long sum ≥ 7: {win_rate_sum_7_or_greater:.2f}%")
print(f"Average percentage of won trades: {average_percentage_won_sum_7_or_greater:.2f}%")
print(f"Average percentage of lost trades: {average_percentage_lost_sum_7_or_greater:.2f}%")

trades_sum_less_than_7 = df_trades_complete[(df_trades_complete['Medium'] + df_trades_complete['Long']) < 7]

count_trades_sum_less_than_7 = trades_sum_less_than_7.shape[0]

trades_won_sum_less_than_7 = trades_sum_less_than_7[trades_sum_less_than_7['Percentage'] > 0]
trades_lost_sum_less_than_7 = trades_sum_less_than_7[trades_sum_less_than_7['Percentage'] <= 0]

win_rate_sum_less_than_7 = (len(trades_won_sum_less_than_7) / len(trades_sum_less_than_7)) * 100 if len(trades_sum_less_than_7) > 0 else 0

average_percentage_won_sum_less_than_7 = trades_won_sum_less_than_7['Percentage'].mean() * 100 if len(trades_won_sum_less_than_7) > 0 else 0
average_percentage_lost_sum_less_than_7 = trades_lost_sum_less_than_7['Percentage'].mean() * 100 if len(trades_lost_sum_less_than_7) > 0 else 0

trades_sum_6_or_greater = df_trades_complete[(df_trades_complete['Medium'] + df_trades_complete['Long']) >= 6]

count_trades_sum_6_or_greater = trades_sum_6_or_greater.shape[0]

trades_won_sum_6_or_greater = trades_sum_6_or_greater[trades_sum_6_or_greater['Percentage'] > 0]
trades_lost_sum_6_or_greater = trades_sum_6_or_greater[trades_sum_6_or_greater['Percentage'] <= 0]

win_rate_sum_6_or_greater = (len(trades_won_sum_6_or_greater) / len(trades_sum_6_or_greater)) * 100 if len(trades_sum_6_or_greater) > 0 else 0

average_percentage_won_sum_6_or_greater = trades_won_sum_6_or_greater['Percentage'].mean() * 100 if len(trades_won_sum_6_or_greater) > 0 else 0
average_percentage_lost_sum_6_or_greater = trades_lost_sum_6_or_greater['Percentage'].mean() * 100 if len(trades_lost_sum_6_or_greater) > 0 else 0

print()
print(f"Number of completed trades with Medium + Long sum ≥ 6: {count_trades_sum_6_or_greater}")
print(f"Win rate for trades with Medium + Long sum ≥ 6: {win_rate_sum_6_or_greater:.2f}%")
print(f"Average percentage of won trades: {average_percentage_won_sum_6_or_greater:.2f}%")
print(f"Average percentage of lost trades: {average_percentage_lost_sum_6_or_greater:.2f}%")

trades_sum_5_or_greater = df_trades_complete[(df_trades_complete['Medium'] + df_trades_complete['Long']) >= 5]

count_trades_sum_5_or_greater = trades_sum_5_or_greater.shape[0]

trades_won_sum_5_or_greater = trades_sum_5_or_greater[trades_sum_5_or_greater['Percentage'] > 0]
trades_lost_sum_5_or_greater = trades_sum_5_or_greater[trades_sum_5_or_greater['Percentage'] <= 0]

win_rate_sum_5_or_greater = (len(trades_won_sum_5_or_greater) / len(trades_sum_5_or_greater)) * 100 if len(trades_sum_5_or_greater) > 0 else 0

average_percentage_won_sum_5_or_greater = trades_won_sum_5_or_greater['Percentage'].mean() * 100 if len(trades_won_sum_5_or_greater) > 0 else 0
average_percentage_lost_sum_5_or_greater = trades_lost_sum_5_or_greater['Percentage'].mean() * 100 if len(trades_lost_sum_5_or_greater) > 0 else 0

print()
print(f"Number of completed trades with Medium + Long sum ≥ 5: {count_trades_sum_5_or_greater}")
print(f"Win rate for trades with Medium + Long sum ≥ 5: {win_rate_sum_5_or_greater:.2f}%")
print(f"Average percentage of won trades: {average_percentage_won_sum_5_or_greater:.2f}%")
print(f"Average percentage of lost trades: {average_percentage_lost_sum_5_or_greater:.2f}%")

trades_sum_exactly_8_SM = df_trades_complete[(df_trades_complete['Short'] + df_trades_complete['Medium']) == 8]

count_trades_sum_exactly_8_SM = trades_sum_exactly_8_SM.shape[0]

trades_won_sum_exactly_8_SM = trades_sum_exactly_8_SM[trades_sum_exactly_8_SM['Percentage'] > 0]
trades_lost_sum_exactly_8_SM = trades_sum_exactly_8_SM[trades_sum_exactly_8_SM['Percentage'] <= 0]

win_rate_sum_exactly_8_SM = (len(trades_won_sum_exactly_8_SM) / len(trades_sum_exactly_8_SM)) * 100 if len(trades_sum_exactly_8_SM) > 0 else 0

average_percentage_won_sum_exactly_8_SM = trades_won_sum_exactly_8_SM['Percentage'].mean() * 100 if len(trades_won_sum_exactly_8_SM) > 0 else 0
average_percentage_lost_sum_exactly_8_SM = trades_lost_sum_exactly_8_SM['Percentage'].mean() * 100 if len(trades_lost_sum_exactly_8_SM) > 0 else 0

print()
print(f"Number of completed trades with Short + Medium sum = 8: {count_trades_sum_exactly_8_SM}")
print(f"Win rate for trades with Short + Medium sum = 8: {win_rate_sum_exactly_8_SM:.2f}%")
print(f"Average percentage of won trades: {average_percentage_won_sum_exactly_8_SM:.2f}%")
print(f"Average percentage of lost trades: {average_percentage_lost_sum_exactly_8_SM:.2f}%")

trades_sum_7_or_greater_SM = df_trades_complete[(df_trades_complete['Short'] + df_trades_complete['Medium']) >= 7]

count_trades_sum_7_or_greater_SM = trades_sum_7_or_greater_SM.shape[0]

trades_won_sum_7_or_greater_SM = trades_sum_7_or_greater_SM[trades_sum_7_or_greater_SM['Percentage'] > 0]
trades_lost_sum_7_or_greater_SM = trades_sum_7_or_greater_SM[trades_sum_7_or_greater_SM['Percentage'] <= 0]

win_rate_sum_7_or_greater_SM = (len(trades_won_sum_7_or_greater_SM) / len(trades_sum_7_or_greater_SM)) * 100 if len(trades_sum_7_or_greater_SM) > 0 else 0

average_percentage_won_sum_7_or_greater_SM = trades_won_sum_7_or_greater_SM['Percentage'].mean() * 100 if len(trades_won_sum_7_or_greater_SM) > 0 else 0
average_percentage_lost_sum_7_or_greater_SM = trades_lost_sum_7_or_greater_SM['Percentage'].mean() * 100 if len(trades_lost_sum_7_or_greater_SM) > 0 else 0

print()
print(f"Number of completed trades with Short + Medium sum ≥ 7: {count_trades_sum_7_or_greater_SM}")
print(f"Win rate for trades with Short + Medium sum ≥ 7: {win_rate_sum_7_or_greater_SM:.2f}%")
print(f"Average percentage of won trades: {average_percentage_won_sum_7_or_greater_SM:.2f}%")
print(f"Average percentage of lost trades: {average_percentage_lost_sum_7_or_greater_SM:.2f}%")

trades_sum_6_or_greater_SM = df_trades_complete[(df_trades_complete['Short'] + df_trades_complete['Medium']) >= 6]

count_trades_sum_6_or_greater_SM = trades_sum_6_or_greater_SM.shape[0]

trades_won_sum_6_or_greater_SM = trades_sum_6_or_greater_SM[trades_sum_6_or_greater_SM['Percentage'] > 0]
trades_lost_sum_6_or_greater_SM = trades_sum_6_or_greater_SM[trades_sum_6_or_greater_SM['Percentage'] <= 0]

win_rate_sum_6_or_greater_SM = (len(trades_won_sum_6_or_greater_SM) / len(trades_sum_6_or_greater_SM)) * 100 if len(trades_sum_6_or_greater_SM) > 0 else 0

average_percentage_won_sum_6_or_greater_SM = trades_won_sum_6_or_greater_SM['Percentage'].mean() * 100 if len(trades_won_sum_6_or_greater_SM) > 0 else 0
average_percentage_lost_sum_6_or_greater_SM = trades_lost_sum_6_or_greater_SM['Percentage'].mean() * 100 if len(trades_lost_sum_6_or_greater_SM) > 0 else 0

print()
print(f"Number of completed trades with Short + Medium sum ≥ 6: {count_trades_sum_6_or_greater_SM}")
print(f"Win rate for trades with Short + Medium sum ≥ 6: {win_rate_sum_6_or_greater_SM:.2f}%")
print(f"Average percentage of won trades: {average_percentage_won_sum_6_or_greater_SM:.2f}%")
print(f"Average percentage of lost trades: {average_percentage_lost_sum_6_or_greater_SM:.2f}%")

trades_sum_5_or_greater_SM = df_trades_complete[(df_trades_complete['Short'] + df_trades_complete['Medium']) >= 5]

count_trades_sum_5_or_greater_SM = trades_sum_5_or_greater_SM.shape[0]

trades_won_sum_5_or_greater_SM = trades_sum_5_or_greater_SM[trades_sum_5_or_greater_SM['Percentage'] > 0]
trades_lost_sum_5_or_greater_SM = trades_sum_5_or_greater_SM[trades_sum_5_or_greater_SM['Percentage'] <= 0]

win_rate_sum_5_or_greater_SM = (len(trades_won_sum_5_or_greater_SM) / len(trades_sum_5_or_greater_SM)) * 100 if len(trades_sum_5_or_greater_SM) > 0 else 0

average_percentage_won_sum_5_or_greater_SM = trades_won_sum_5_or_greater_SM['Percentage'].mean() * 100 if len(trades_won_sum_5_or_greater_SM) > 0 else 0
average_percentage_lost_sum_5_or_greater_SM = trades_lost_sum_5_or_greater_SM['Percentage'].mean() * 100 if len(trades_lost_sum_5_or_greater_SM) > 0 else 0

print()
print(f"Number of completed trades with Short + Medium sum ≥ 5: {count_trades_sum_5_or_greater_SM}")
print(f"Win rate for trades with Short + Medium sum ≥ 5: {win_rate_sum_5_or_greater_SM:.2f}%")
print(f"Average percentage of won trades: {average_percentage_won_sum_5_or_greater_SM:.2f}%")
print(f"Average percentage of lost trades: {average_percentage_lost_sum_5_or_greater_SM:.2f}%")

trades_sum_exactly_8_SL = df_trades_complete[(df_trades_complete['Short'] + df_trades_complete['Long']) == 8]

count_trades_sum_exactly_8_SL = trades_sum_exactly_8_SL.shape[0]

trades_won_sum_exactly_8_SL = trades_sum_exactly_8_SL[trades_sum_exactly_8_SL['Percentage'] > 0]
trades_lost_sum_exactly_8_SL = trades_sum_exactly_8_SL[trades_sum_exactly_8_SL['Percentage'] <= 0]

win_rate_sum_exactly_8_SL = (len(trades_won_sum_exactly_8_SL) / len(trades_sum_exactly_8_SL)) * 100 if len(trades_sum_exactly_8_SL) > 0 else 0

average_percentage_won_sum_exactly_8_SL = trades_won_sum_exactly_8_SL['Percentage'].mean() * 100 if len(trades_won_sum_exactly_8_SL) > 0 else 0
average_percentage_lost_sum_exactly_8_SL = trades_lost_sum_exactly_8_SL['Percentage'].mean() * 100 if len(trades_lost_sum_exactly_8_SL) > 0 else 0

print()
print(f"Number of completed trades with Short + Long sum = 8: {count_trades_sum_exactly_8_SL}")
print(f"Win rate for trades with Short + Long sum = 8: {win_rate_sum_exactly_8_SL:.2f}%")
print(f"Average percentage of won trades: {average_percentage_won_sum_exactly_8_SL:.2f}%")
print(f"Average percentage of lost trades: {average_percentage_lost_sum_exactly_8_SL:.2f}%")

trades_sum_7_or_greater_SL = df_trades_complete[(df_trades_complete['Short'] + df_trades_complete['Long']) >= 7]

count_trades_sum_7_or_greater_SL = trades_sum_7_or_greater_SL.shape[0]

trades_won_sum_7_or_greater_SL = trades_sum_7_or_greater_SL[trades_sum_7_or_greater_SL['Percentage'] > 0]
trades_lost_sum_7_or_greater_SL = trades_sum_7_or_greater_SL[trades_sum_7_or_greater_SL['Percentage'] <= 0]

win_rate_sum_7_or_greater_SL = (len(trades_won_sum_7_or_greater_SL) / len(trades_sum_7_or_greater_SL)) * 100 if len(trades_sum_7_or_greater_SL) > 0 else 0

average_percentage_won_sum_7_or_greater_SL = trades_won_sum_7_or_greater_SL['Percentage'].mean() * 100 if len(trades_won_sum_7_or_greater_SL) > 0 else 0
average_percentage_lost_sum_7_or_greater_SL = trades_lost_sum_7_or_greater_SL['Percentage'].mean() * 100 if len(trades_lost_sum_7_or_greater_SL) > 0 else 0

print()
print(f"Number of completed trades with Short + Long sum ≥ 7: {count_trades_sum_7_or_greater_SL}")
print(f"Win rate for trades with Short + Long sum ≥ 7: {win_rate_sum_7_or_greater_SL:.2f}%")
print(f"Average percentage of won trades: {average_percentage_won_sum_7_or_greater_SL:.2f}%")
print(f"Average percentage of lost trades: {average_percentage_lost_sum_7_or_greater_SL:.2f}%")

trades_sum_6_or_greater_SL = df_trades_complete[(df_trades_complete['Short'] + df_trades_complete['Long']) >= 6]

count_trades_sum_6_or_greater_SL = trades_sum_6_or_greater_SL.shape[0]

trades_won_sum_6_or_greater_SL = trades_sum_6_or_greater_SL[trades_sum_6_or_greater_SL['Percentage'] > 0]
trades_lost_sum_6_or_greater_SL = trades_sum_6_or_greater_SL[trades_sum_6_or_greater_SL['Percentage'] <= 0]

win_rate_sum_6_or_greater_SL = (len(trades_won_sum_6_or_greater_SL) / len(trades_sum_6_or_greater_SL)) * 100 if len(trades_sum_6_or_greater_SL) > 0 else 0

average_percentage_won_sum_6_or_greater_SL = trades_won_sum_6_or_greater_SL['Percentage'].mean() * 100 if len(trades_won_sum_6_or_greater_SL) > 0 else 0
average_percentage_lost_sum_6_or_greater_SL = trades_lost_sum_6_or_greater_SL['Percentage'].mean() * 100 if len(trades_lost_sum_6_or_greater_SL) > 0 else 0

print()
print(f"Number of completed trades with Short + Long sum ≥ 6: {count_trades_sum_6_or_greater_SL}")
print(f"Win rate for trades with Short + Long sum ≥ 6: {win_rate_sum_6_or_greater_SL:.2f}%")
print(f"Average percentage of won trades: {average_percentage_won_sum_6_or_greater_SL:.2f}%")
print(f"Average percentage of lost trades: {average_percentage_lost_sum_6_or_greater_SL:.2f}%")

trades_sum_5_or_greater_SL = df_trades_complete[(df_trades_complete['Short'] + df_trades_complete['Long']) >= 5]

count_trades_sum_5_or_greater_SL = trades_sum_5_or_greater_SL.shape[0]

trades_won_sum_5_or_greater_SL = trades_sum_5_or_greater_SL[trades_sum_5_or_greater_SL['Percentage'] > 0]
trades_lost_sum_5_or_greater_SL = trades_sum_5_or_greater_SL[trades_sum_5_or_greater_SL['Percentage'] <= 0]

win_rate_sum_5_or_greater_SL = (len(trades_won_sum_5_or_greater_SL) / len(trades_sum_5_or_greater_SL)) * 100 if len(trades_sum_5_or_greater_SL) > 0 else 0

average_percentage_won_sum_5_or_greater_SL = trades_won_sum_5_or_greater_SL['Percentage'].mean() * 100 if len(trades_won_sum_5_or_greater_SL) > 0 else 0
average_percentage_lost_sum_5_or_greater_SL = trades_lost_sum_5_or_greater_SL['Percentage'].mean() * 100 if len(trades_lost_sum_5_or_greater_SL) > 0 else 0

print()
print(f"Number of completed trades with Short + Long sum ≥ 5: {count_trades_sum_5_or_greater_SL}")
print(f"Win rate for trades with Short + Long sum ≥ 5: {win_rate_sum_5_or_greater_SL:.2f}%")
print(f"Average percentage of won trades: {average_percentage_won_sum_5_or_greater_SL:.2f}%")
print(f"Average percentage of lost trades: {average_percentage_lost_sum_5_or_greater_SL:.2f}%")

df_trades_complete['Percentage'] = pd.to_numeric(df_trades_complete['Percentage'], errors='coerce')

df_trades_complete['Trade Won'] = df_trades_complete['Percentage'] > 0

plt.figure(figsize=(10, 6))

plt.scatter(df_trades_complete['Market Cap Numeric'], df_trades_complete['Percentage'],
            c=df_trades_complete['Trade Won'].map({True: 'green', False: 'red'}),
            label='Trade Won (Green) / Lost (Red)')

plt.title('Market Cap vs. Percentage (Completed Trades)')
plt.xlabel('Market Cap (Numeric)')
plt.ylabel('Percentage')
plt.xscale('log')

plt.gca().xaxis.set_major_formatter(FuncFormatter(format_market_cap))
plt.grid(True)

plt.legend()

plt.show()

plt.figure(figsize=(10, 6))

plt.scatter(df_trades_complete['Business Days'], df_trades_complete['Percentage'],
            c=df_trades_complete['Trade Won'].map({True: 'green', False: 'red'}),
            alpha=0.7,
            edgecolors='w',
            linewidth=0.5)

plt.title('Trade Duration vs. Percentage (Completed Trades)')
plt.xlabel('Business Days')
plt.ylabel('Percentage')

plt.axhline(0, color='grey', lw=0.8)
plt.grid(True, which='both', linestyle='--', linewidth=0.5)
plt.tight_layout()

plt.show()

# plt.scatter(df_trades_complete['Market Cap Numeric'], df_trades_complete['Business Days'],
#             c=df_trades_complete['Trade Won'].map({True: 'blue', False: 'orange'}),
#             alpha=0.7,
#             edgecolors='w',
#             linewidth=0.5)

# plt.title('Market Cap vs. Trade Duration (Completed Trades)')
# plt.ylabel('Business Days')
# plt.xlabel('Market Cap')

# plt.xscale('log')
# plt.yscale('linear')

# plt.gca().xaxis.set_major_formatter(FuncFormatter(format_market_cap))

# plt.grid(True, which='both', linestyle='--', linewidth=0.5)
# plt.tight_layout()

# plt.show()

trade_duration_counts = df_trades_complete['Business Days'].value_counts().sort_index()

trade_duration_analysis = df_trades_complete.groupby('Business Days').agg(
    Trades_Count=('Trade Won', 'size'),
    Win_Rate=('Trade Won', lambda x: (x.sum() / x.count()) * 100)
).reset_index()

print(trade_duration_analysis)

trades_5_days_or_more = df_trades_complete[df_trades_complete['Business Days'] >= 5]
win_rate_5_days_or_more = (trades_5_days_or_more['Trade Won'].sum() / len(trades_5_days_or_more)) * 100 if len(trades_5_days_or_more) > 0 else 0

trades_4_days_or_less = df_trades_complete[df_trades_complete['Business Days'] <= 4]
win_rate_4_days_or_less = (trades_4_days_or_less['Trade Won'].sum() / len(trades_4_days_or_less)) * 100 if len(trades_4_days_or_less) > 0 else 0

print()
print(f"Win rate for trades lasting 5 days or longer: {win_rate_5_days_or_more:.2f}%")
print(f"Win rate for trades lasting 4 days or less: {win_rate_4_days_or_less:.2f}%")

df_trades_complete['Trade Outcome'] = df_trades_complete['Percentage'].apply(lambda x: 'Winning' if x > 0 else 'Losing')

average_values_by_outcome = df_trades_complete.groupby('Trade Outcome')[['Short', 'Medium', 'Long', 'Total Pts']].mean()

print()
print(average_values_by_outcome)

stocks_200M_to_1B = df_trades_complete[(df_trades_complete['Market Cap Numeric'] >= 200e6) & (df_trades_complete['Market Cap Numeric'] <= 1e9)]

avg_values_200M_to_1B = stocks_200M_to_1B[['Short', 'Medium', 'Long', 'Total Pts']].mean()

stocks_over_1B = df_trades_complete[df_trades_complete['Market Cap Numeric'] > 1e9]

avg_values_over_1B = stocks_over_1B[['Short', 'Medium', 'Long', 'Total Pts']].mean()

total_pts_counts_complete = df_trades_complete['Total Pts'].value_counts().sort_index()

print()
print("Total Completed Trades by", total_pts_counts_complete)

win_rate_by_total_pts = df_trades_complete.groupby('Total Pts')['Trade Won'].mean() * 100

print("Win Rate by", win_rate_by_total_pts)

average_percentage_by_total_pts = df_trades_complete.groupby('Total Pts').agg(
    Average_Percentage_Won=('Percentage', lambda x: x[x > 0].mean()),
    Average_Percentage_Lost=('Percentage', lambda x: x[x <= 0].mean())
)

average_percentage_by_total_pts['Average_Percentage_Won'] = average_percentage_by_total_pts['Average_Percentage_Won'] * 100
average_percentage_by_total_pts['Average_Percentage_Lost'] = average_percentage_by_total_pts['Average_Percentage_Lost'] * 100

print(average_percentage_by_total_pts)

df_trades_complete['Date Bought'] = pd.to_datetime(df_trades_complete['Date Bought'], format='%m/%d/%Y')

cutoff_date = pd.Timestamp('2024-03-29')

trades_before = df_trades_complete[df_trades_complete['Date Bought'] < cutoff_date]
trades_after= df_trades_complete[df_trades_complete['Date Bought'] >= cutoff_date]

total_trades_before = trades_before.shape[0]
total_trades_after = trades_after.shape[0]

wins_before = trades_before['Trade Won'].sum()
wins_after = trades_after['Trade Won'].sum()

win_rate_before = (wins_before / total_trades_before * 100) if total_trades_before > 0 else 0
win_rate_after = (wins_after / total_trades_after * 100) if total_trades_after > 0 else 0

print(f"Total trades before {cutoff_date.date()}: {total_trades_before}")
print(f"Total trades on or after {cutoff_date.date()}: {total_trades_after}")
print(f"Win rate before {cutoff_date.date()}: {win_rate_before:.2f}%")
print(f"Win rate on or after {cutoff_date.date()}: {win_rate_after:.2f}%")

filtered_trades = df_trades_complete[df_trades_complete['Total Pts'].isin([11, 12])]

def format_market_cap(x, pos):
    if x >= 1e9:
        return '{:.1f}B'.format(x * 1e-9)
    elif x >= 1e6:
        return '{:.1f}M'.format(x * 1e-6)
    elif x >= 1e3:
        return '{:.1f}K'.format(x * 1e-3)
    else:
        return str(x)

plt.figure(figsize=(10, 6))
plt.scatter(filtered_trades['Market Cap Numeric'], filtered_trades['Percentage'],
            c=filtered_trades['Trade Won'].map({True: 'green', False: 'red'}),
            label='Trade Won (Green) / Lost (Red)')

plt.title('Market Cap vs. Percentage for Total Pts of 11 or 12')
plt.xlabel('Market Cap (Numeric)')
plt.ylabel('Percentage')
plt.xscale('log')
plt.gca().xaxis.set_major_formatter(FuncFormatter(format_market_cap))
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()

filtered_trades_10 = df_trades_complete[df_trades_complete['Total Pts'] == 10]

plt.figure(figsize=(10, 6))
plt.scatter(filtered_trades_10['Market Cap Numeric'], filtered_trades_10['Percentage'],
            c=filtered_trades_10['Trade Won'].map({True: 'green', False: 'red'}),
            label='Trade Won (Green) / Lost (Red)')
plt.title('Market Cap vs. Percentage for Total Pts of 10')
plt.xlabel('Market Cap (Numeric)')
plt.ylabel('Percentage')
plt.xscale('log')
plt.gca().xaxis.set_major_formatter(FuncFormatter(format_market_cap))
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()

filtered_trades_9 = df_trades_complete[df_trades_complete['Total Pts'] == 9]

plt.figure(figsize=(10, 6))
plt.scatter(filtered_trades_9['Market Cap Numeric'], filtered_trades_9['Percentage'],
            c=filtered_trades_9['Trade Won'].map({True: 'green', False: 'red'}),
            label='Trade Won (Green) / Lost (Red)')
plt.title('Market Cap vs. Percentage for Total Pts of 9')
plt.xlabel('Market Cap (Numeric)')
plt.ylabel('Percentage')
plt.xscale('log')
plt.gca().xaxis.set_major_formatter(FuncFormatter(format_market_cap))
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()


In [ ]:
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
import numpy as np

file_path = Path('../data/paper_trading_journal.xlsx')
df_newsystem = pd.read_excel(file_path, sheet_name=1)

df_completed_trades = df_newsystem.dropna(subset=['Percentage'])

total_count = df_completed_trades.shape[0]

positive_count = df_completed_trades[df_completed_trades['Percentage'] > 0].shape[0]

non_positive_count = df_completed_trades[df_completed_trades['Percentage'] <= 0].shape[0]

if total_count > 0: 
    percentage_positive_trades = (positive_count / total_count) * 100
else:
    percentage_positive_trades = 0

print("Total Trades Count:", total_count)
print("Positive Percentage Trades Count:", positive_count)
print("Zero or Negative Percentage Trades Count:", non_positive_count)
print("Percentage of Positive Trades:", percentage_positive_trades, "%")

average_winning_percentage = df_completed_trades[df_completed_trades['Percentage'] > 0]['Percentage'].mean()

average_losing_percentage = df_completed_trades[df_completed_trades['Percentage'] <= 0]['Percentage'].mean()

print("Average Winning Trade Percentage:", average_winning_percentage)
print("Average Losing Trade Percentage:", average_losing_percentage)

plt.figure(figsize=(10, 6))
plt.scatter(df_completed_trades['Total Pts'], df_completed_trades['Percentage'], color='blue', alpha=0.5)  # Scatter plot
plt.title('Scatterplot of Total Points vs Percentage')
plt.xlabel('Total Points (Total Pts)')
plt.ylabel('Percentage')
plt.grid(True)
plt.xlim(0, 12)

plt.show()

df_positive_rows = df_completed_trades[(df_completed_trades['Short'] >= 0) &
                                       (df_completed_trades['Medium'] >= 0) &
                                       (df_completed_trades['Long'] >= 0)]

positive_rows_count = df_positive_rows.shape[0]

other_rows_count = df_completed_trades.shape[0] - positive_rows_count

print(f"Count of rows with all values 0 or higher: {positive_rows_count}")
print(f"Count of all other rows: {other_rows_count}")

group_positive_criteria = (df_completed_trades['Short'] >= 0) & (df_completed_trades['Medium'] >= 0) & (df_completed_trades['Long'] >= 0)

group_positive = df_completed_trades[group_positive_criteria]
group_other = df_completed_trades[~group_positive_criteria]

win_rate_positive = (group_positive['Percentage'] > 0).mean() * 100
win_rate_other = (group_other['Percentage'] > 0).mean() * 100

print(f"Win rate for group with all values 0 or higher: {win_rate_positive:.2f}%")
print(f"Win rate for all other groups: {win_rate_other:.2f}%")

average_percentage_positive = group_positive['Percentage'].mean()
average_percentage_other = group_other['Percentage'].mean()

print(f"Average Percentage for group with all values 0 or higher: {average_percentage_positive:.3f}%")
print(f"Average Percentage for all other groups: {average_percentage_other:.3f}%")

average_winning_positive = group_positive[group_positive['Percentage'] > 0]['Percentage'].mean()
average_losing_positive = group_positive[group_positive['Percentage'] <= 0]['Percentage'].mean()
average_winning_other = group_other[group_other['Percentage'] > 0]['Percentage'].mean()
average_losing_other = group_other[group_other['Percentage'] <= 0]['Percentage'].mean()

print("For group with all values 0 or higher:")
print(f"  Average Winning Trade Percentage: {average_winning_positive:.3f}%")
print(f"  Average Losing Trade Percentage: {average_losing_positive:.3f}%")
print("For all other groups:")
print(f"  Average Winning Trade Percentage: {average_winning_other:.3f}%")
print(f"  Average Losing Trade Percentage: {average_losing_other:.3f}%")
completed_trades_count_by_total_pts = df_completed_trades.groupby('Total Pts').size()

print()
print(completed_trades_count_by_total_pts)

grouped_by_total_pts = df_completed_trades.groupby('Total Pts')

def calculate_win_rate(group):
    return (group['Percentage'] > 0).mean() * 100

win_rate_by_total_pts = grouped_by_total_pts.apply(calculate_win_rate)

print(win_rate_by_total_pts)